In [7]:
# Import necessary libraries
import os
import re
import numpy as np
import pandas as pd
from PyPDF2 import PdfReader
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from joblib import dump, load


# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/hugo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/hugo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:

def extract_text_from_pdfs(directory):
    """
    Extracts text from all PDF files in a directory and saves them in a DataFrame.
    
    Parameters:
    directory (str): Path to the directory containing PDF files
    
    Returns:
    pandas.DataFrame: DataFrame with filenames and extracted text
    """
    texts = []
    filenames = []
    
    # Iterate through all files in the directory
    for file in os.listdir(directory):
        if file.endswith('.pdf'):
            full_path = os.path.join(directory, file)
            
            try:
                # Open the PDF
                reader = PdfReader(full_path)
                full_text = ""
                
                # Extract text from each page
                for page in reader.pages:
                    full_text += page.extract_text() + " "
                
                # Save the text and filename
                texts.append(full_text)
                filenames.append(file)
                print(f"Successfully processed: {file}")
            except Exception as e:
                print(f"Error processing {file}: {e}")
    
    # Create a DataFrame with extracted texts
    df_texts = pd.DataFrame({
        'filename': filenames,
        'text': texts
    })
    
    return df_texts

# Replace "path/to/your/pdfs" with the actual path to your PDF files


In [9]:
def preprocess_text(text):
    """
    Cleans and preprocesses text: removes special characters,
    converts to lowercase, tokenizes, removes stopwords, and applies stemming.
    
    Parameters:
    text (str): The text to preprocess
    
    Returns:
    str: Preprocessed text
    """
    # Configure stemmer for Spanish
    stemmer = SnowballStemmer('spanish')
    stop_words = set(stopwords.words('spanish'))
    
    # Convert to lowercase and remove special characters
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text) # Remove special characters
    
    # Tokenize the text
    tokens = nltk.word_tokenize(text, language='spanish')
    
    # Remove stopwords and apply stemming
    processed_tokens = [stemmer.stem(token) for token in tokens if token not in stop_words]
    
    return ' '.join(processed_tokens)

In [10]:
def create_qa_corpus():
    """
    Creates a corpus of questions and answers based on breast cancer medical information in Spanish.
    
    Returns:
    pandas.DataFrame: DataFrame containing question-answer pairs with categories
    """
    # This is a simplified version. In a real project,
    # you would segment the documents into thematic sections
    # and generate relevant question-answer pairs.
    
    corpus_qa = [
        # Examples of question-answer pairs with their categories
        {"pregunta": "¿Qué es el cáncer de mama?", 
         "respuesta": "El cáncer de mama es una enfermedad en la que las células del tejido mamario crecen de forma descontrolada. Existen diferentes tipos de cáncer de mama dependiendo de qué células de la mama se convierten en cancerosas.", 
         "categoria": "definicion"},
        
        {"pregunta": "¿Cuáles son los síntomas del cáncer de mama?", 
         "respuesta": "Los síntomas del cáncer de mama pueden incluir un bulto en el seno, secreción sanguinolenta del pezón y cambios en la forma o textura del pezón o del seno.", 
         "categoria": "sintomas"},
        
        {"pregunta": "¿Cómo se diagnostica el cáncer de mama?", 
         "respuesta": "El cáncer de mama se puede diagnosticar mediante exámenes clínicos de las mamas, mamografías, ultrasonidos, resonancias magnéticas y biopsias de áreas sospechosas.", 
         "categoria": "diagnostico"},
        
        {"pregunta": "¿Cuáles son los factores de riesgo para el cáncer de mama?", 
         "respuesta": "Los factores de riesgo incluyen edad, mutaciones genéticas, antecedentes familiares, antecedentes personales de cáncer de mama, tejido mamario denso y ciertos factores de estilo de vida.", 
         "categoria": "factores_riesgo"},
        
        {"pregunta": "¿Qué tratamientos están disponibles para el cáncer de mama?", 
         "respuesta": "Los tratamientos pueden incluir cirugía, radioterapia, quimioterapia, terapia hormonal, terapia dirigida e inmunoterapia.", 
         "categoria": "tratamiento"},
        
        {"pregunta": "¿Cuáles son las etapas del cáncer de mama?", 
         "respuesta": "Las etapas del cáncer de mama van de 0 a IV, siendo la etapa 0 un cáncer no invasivo y la etapa IV un cáncer metastásico que se ha extendido a otras partes del cuerpo.", 
         "categoria": "etapas"},
        
        # Add more examples based on sections in your documents
    ]
    
    return pd.DataFrame(corpus_qa)

In [11]:
def train_naive_bayes_model(corpus_qa):
    """
    Trains a Naive Bayes model to classify questions into categories.
    
    Parameters:
    corpus_qa (pandas.DataFrame): DataFrame with question-answer pairs
    
    Returns:
    sklearn.pipeline.Pipeline: Trained model pipeline
    """
    # Prepare the data
    X = corpus_qa['pregunta'].apply(preprocess_text)
    y = corpus_qa['categoria']
    
    # Split into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Create a processing and classification pipeline
    pipeline = Pipeline([
        ('vectorizer', TfidfVectorizer(max_features=5000)),
        ('classifier', MultinomialNB())
    ])
    
    # Train the model
    pipeline.fit(X_train, y_train)
    
    # Evaluate the model
    accuracy = pipeline.score(X_test, y_test)
    print(f"Precisión del modelo: {accuracy:.2f}")
    
    return pipeline



In [12]:
class ChatbotCancerMama:
    def __init__(self, ruta_modelo, corpus_qa):
        """
        Initializes the chatbot by loading the trained model and response corpus.
        
        Parameters:
        ruta_modelo (str): Path to the saved model
        corpus_qa (pandas.DataFrame): DataFrame with question-answer pairs
        """
        self.modelo = load(ruta_modelo)
        self.corpus_qa = corpus_qa
        
        # Create a dictionary of responses by category
        self.respuestas_por_categoria = {}
        for _, fila in corpus_qa.iterrows():
            categoria = fila['categoria']
            respuesta = fila['respuesta']
            
            if categoria not in self.respuestas_por_categoria:
                self.respuestas_por_categoria[categoria] = []
            
            self.respuestas_por_categoria[categoria].append(respuesta)
    
    def obtener_respuesta(self, pregunta):
        """
        Processes the user's question and returns an appropriate response.
        
        Parameters:
        pregunta (str): User's question
        
        Returns:
        str: Chatbot's response
        """
        # Preprocess the question
        pregunta_procesada = preprocess_text(pregunta)
        
        # Predict the category
        categoria_predicha = self.modelo.predict([pregunta_procesada])[0]
        
        # Get a response from that category
        # In a more advanced system, you could implement a more sophisticated selection
        import random
        respuestas_disponibles = self.respuestas_por_categoria.get(categoria_predicha, 
            ["Lo siento, no tengo información sobre eso."])
        
        return random.choice(respuestas_disponibles)

# Example usage
# corpus_qa = pd.read_csv("corpus_qa_cancer_mama.csv")
# chatbot = ChatbotCancerMama('modelo_chatbot_cancer_mama.joblib', corpus_qa)

def iniciar_chat(chatbot):
    """
    Starts an interactive chat session with the chatbot.
    
    Parameters:
    chatbot (ChatbotCancerMama): An initialized chatbot instance
    """
    print("Chatbot de Asesoramiento sobre Cáncer de Mama")
    print("Escribe 'salir' para terminar la conversación.")
    
    while True:
        pregunta = input("\nTú: ")
        
        if pregunta.lower() == 'salir':
            print("¡Hasta pronto!")
            break
        
        respuesta = chatbot.obtener_respuesta(pregunta)
        print(f"Chatbot: {respuesta}")


In [13]:
def extraer_frases_relevantes(texto, n_frases=3):
    """
    Divides the text into sentences and extracts the most relevant ones.
    
    Parameters:
    texto (str): Text to extract phrases from
    n_frases (int): Number of phrases to extract
    
    Returns:
    list: List of extracted phrases
    """
    # Split text into sentences (simplified)
    frases = re.split(r'[.!?]+', texto)
    frases = [frase.strip() for frase in frases if len(frase.strip()) > 10]
    
    return frases[:n_frases]  # In a real system, you'd use a ranking algorithm

class ChatbotAvanzado(ChatbotCancerMama):
    def __init__(self, ruta_modelo, corpus_qa, df_textos):
        """
        Initializes an enhanced chatbot that can search through the original documents.
        
        Parameters:
        ruta_modelo (str): Path to the saved model
        corpus_qa (pandas.DataFrame): DataFrame with question-answer pairs
        df_textos (pandas.DataFrame): DataFrame with the original document texts
        """
        super().__init__(ruta_modelo, corpus_qa)
        self.df_textos = df_textos
        
        # Vectorize all documents for search
        self.vectorizador = TfidfVectorizer(max_features=5000)
        textos_procesados = df_textos['texto_procesado'].tolist()
        self.matriz_documentos = self.vectorizador.fit_transform(textos_procesados)
        
    def buscar_informacion_adicional(self, pregunta, top_k=2):
        """
        Searches for additional information in the original documents.
        
        Parameters:
        pregunta (str): User's question
        top_k (int): Number of most relevant documents to consider
        
        Returns:
        str: Additional information from relevant documents
        """
        # Vectorize the question
        vector_pregunta = self.vectorizador.transform([preprocess_text(pregunta)])
        
        # Calculate similarity with documents
        similitudes = cosine_similarity(vector_pregunta, self.matriz_documentos)[0]
        
        # Find the most relevant documents
        indices_relevantes = np.argsort(similitudes)[-top_k:]
        
        # Extract relevant phrases from those documents
        informacion_adicional = []
        for idx in indices_relevantes:
            texto_completo = self.df_textos.iloc[idx]['texto']
            frases = extraer_frases_relevantes(texto_completo)
            informacion_adicional.extend(frases)
        
        return "\n".join(informacion_adicional)
    
    def obtener_respuesta(self, pregunta):
        """
        Enhanced version that combines classification with information search.
        
        Parameters:
        pregunta (str): User's question
        
        Returns:
        str: Chatbot's response with additional information
        """
        respuesta_base = super().obtener_respuesta(pregunta)
        info_adicional = self.buscar_informacion_adicional(pregunta)
        
        return f"{respuesta_base}\n\nInformación adicional:\n{info_adicional}"

In [14]:
# Load the necessary data and initialize the chatbot
def probar_chatbot():
    """
    Function to test the chatbot with some sample questions.
    """
    # In a real notebook, you would load your actual saved files
    try:
        df_textos = pd.read_csv("textos_cancer_mama.csv")
        corpus_qa = pd.read_csv("corpus_qa_cancer_mama.csv")
        
        # Apply preprocessing to the texts if needed
        if 'texto_procesado' not in df_textos.columns:
            print("Preprocesando textos...")
            df_textos['texto_procesado'] = df_textos['texto'].apply(preprocess_text)
            
        # Initialize the advanced chatbot
        chatbot = ChatbotAvanzado('modelo_chatbot_cancer_mama.joblib', corpus_qa, df_textos)
        
        # Test with some sample questions
        preguntas_prueba = [
            "¿Qué es el cáncer de mama?",
            "¿Qué síntomas debo vigilar?",
            "¿Cómo se diagnostica el cáncer de mama?",
            "¿Cuáles son las opciones de tratamiento?",
            "¿Estoy en riesgo de tener cáncer de mama?"
        ]
        
        print("=== Resultados de Prueba del Chatbot ===")
        for pregunta in preguntas_prueba:
            print(f"\nPregunta: {pregunta}")
            respuesta = chatbot.obtener_respuesta(pregunta)
            print(f"Respuesta: {respuesta}")
            
        print("\n=== Chat Interactivo ===")
        print("Ahora puedes hacer tus propias preguntas (escribe 'salir' para terminar):")
        
        while True:
            pregunta_usuario = input("\nTú: ")
            if pregunta_usuario.lower() == 'salir':
                print("¡Hasta pronto!")
                break
            
            respuesta = chatbot.obtener_respuesta(pregunta_usuario)
            print(f"Chatbot: {respuesta}")
            
    except FileNotFoundError as e:
        print(f"Error: {e}")
        print("\nPara ejecutar esta prueba, primero necesitas:")
        print("1. Extraer texto de tus archivos PDF")
        print("2. Crear y guardar tu corpus de preguntas y respuestas")
        print("3. Entrenar y guardar tu modelo")
        
# probar_chatbot()

In [ ]:
df = extract_text_from_pdfs('/Volumes/Proyecto Hugo/breast-cancer-analysis/papers/Base de conocimiento del programa')

Successfully processed: 12. Cáncer mamario en hombres autor Gloria Mesa, Gustavo Matute y Manuela Estrada.pdf
Error processing 04. El proceso del cáncer de mama. Valoración, diagnóstico, y planificación de cuidados autor Lorena Gómez Mora.pdf: PyCryptodome is required for AES algorithm
Successfully processed: 15. Protocolo de tratamiento del cáncer de mama autor Hospital Donostia.pdf
Successfully processed: prevencion-factores-riesgo.pdf
Successfully processed: 02. Guía para entender el cáncer de mama autor Rivas, Leticia.pdf
Successfully processed: 06. Cáncer de Mama, Trabajo y Sociedad autor Encarna Aguilar Jiménez, Fernando Bandrés Moya y Luisa Mercedes Capdevila García.pdf
Successfully processed: Guia-Oncosur-de-Cancer-de-Mama.pdf
Successfully processed: ES-Cancer-de-Mama-Guia-para-Pacientes.pdf
Successfully processed: protocolo_cancer_mama_2021.pdf
Successfully processed: guia-descargable-que-es-la-radioterapia.pdf
Successfully processed: Cuestiones frecuentes.pdf
Successfully pro

In [ ]:
directorio_pdfs = "ruta/a/tus/pdfs"  
df_guias_medicas = extract_text_from_pdfs(directorio_pdfs)

# Save the extracted texts to a CSV file for future use
df_guias_medicas.to_csv("textos_cancer_mama.csv", index=False)
